In [7]:
import anndata
import argparse
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import scanpy as sc
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
import sys
import torch
import torch.distributions as dist
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
logvar = torch.Tensor([0.5, 0.2])
mu = torch.Tensor([1, 0.7])


print(-0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1))
print

In [8]:
class scRNADataset(Dataset):
    """
    Class for the scRNA-seq dataset.
    Accepts path to the dataset, fraction of the dataset to be used 
    and transform function.
    Returns single observation from the dataset 
    as a vector of values and a cell type.
    """
    def __init__(self, h5ad_file: str, sample: float, transform=None):
        self.data = sc.read_h5ad(h5ad_file)
        self.cell_type = np.array(self.data.obs.cell_type)
        self.data = self.data.layers['counts'].toarray()
        l_cells = round(len(self.data) * sample)
        self.data = self.data[:l_cells]
        self.data = self.data/np.max(self.data)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        single_cell = self.data[idx]
        single_cell_type = self.cell_type[idx]

        if self.transform:
            single_cell = self.transform(single_cell)

        return single_cell, single_cell_type


def create_dataloader_dict(h5ad_file_train: str, 
                        h5ad_file_test: str, 
                        batch_size: int, 
                        shuffle: bool,
                        sample: float, 
                        transform=None
                        ) -> dict:
    """
    Creates dataloaders for train and test datasets.
    Returns dictionary with dataloaders.
    """
    training_data = scRNADataset(h5ad_file_train, sample=sample, transform=transform)
    test_data = scRNADataset(h5ad_file_test, sample=sample, transform=transform)
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=shuffle)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=shuffle)

    return {'train': train_dataloader,
            'val': test_dataloader}

In [9]:
dicto = create_dataloader_dict('data/SAD2022Z_Project1_GEX_train.h5ad', 'data/SAD2022Z_Project1_GEX_test.h5ad', 32, True, 0.1, transform=torch.from_numpy)

In [17]:
for i,_ in dicto['train']:
    print(i.shape[1])
    break

5000


In [26]:
# Load datasets
train_data = sc.read_h5ad('data/SAD2022Z_Project1_GEX_train.h5ad')
test_data = sc.read_h5ad('data/SAD2022Z_Project1_GEX_test.h5ad')

In [27]:
test_data = test_data.X.toarray()
test_data2 = (test_data - test_data.min())/(test_data.max() - test_data.min())
test_data2

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        3.1140502e-07, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.8967296e-06, 0.0000000e+00],
       [0.0000000e+00, 5.5492176e-08, 0.0000000e+00, ..., 0.0000000e+00,
        3.8289600e-06, 0.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        3.4113307e-06, 0.0000000e+00],
       [0.0000000e+00, 1.4296064e-07, 0.0000000e+00, ..., 0.0000000e+00,
        2.6447719e-06, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        3.7550076e-06, 0.0000000e+00]], dtype=float32)

In [28]:
preprocessing.normalize(test_data)

array([[0.        , 0.        , 0.        , ..., 0.        , 0.12247444,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.02045638,
        0.        ],
       [0.        , 0.00758338, 0.        , ..., 0.        , 0.5232532 ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.32350838,
        0.        ],
       [0.        , 0.01565945, 0.        , ..., 0.        , 0.28969982,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.47407284,
        0.        ]], dtype=float32)

: 

In [3]:
a= test_data.obs.reset_index(drop=True)
b = list(a[a['cell_type'] == 'CD14+ Mono'].index)

In [23]:
a[a['cell_type'] == 'CD14+ Mono']

,GEX_n_genes_by_counts,GEX_pct_counts_mt,GEX_size_factors,GEX_phase,ADT_n_antibodies_by_counts,ADT_total_counts,ADT_iso_count,cell_type,batch,ADT_pseudotime_order,...,DonorID,DonorAge,DonorBMI,DonorBloodType,DonorRace,Ethnicity,DonorGender,QCMeds,DonorSmoker,is_train
0,948,5.113025,1.371097,G2M,139,2278.0,23.0,CD14+ Mono,s3d1,NaN,...,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker,train
3,1811,7.071157,1.387475,G2M,105,707.0,2.0,CD14+ Mono,s2d5,NaN,...,16710,40,27.8,O+,White,HISPANIC OR LATINO,Female,False,Smoker,iid_holdout
6,741,16.082317,0.489712,G2M,95,647.0,4.0,CD14+ Mono,s2d1,NaN,...,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker,train
9,1876,6.420387,1.253895,G2M,121,1084.0,5.0,CD14+ Mono,s3d7,NaN,...,11466,22,31.5,A+,Asian,NOT HISPANIC OR LATINO,Female,True,Nonsmoker,train
17,1498,6.517647,1.003521,G2M,114,641.0,6.0,CD14+ Mono,s3d6,NaN,...,28045,36,23.8,A+,Other Race,HISPANIC OR LATINO,Female,False,Nonsmoker,iid_holdout
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18023,2273,6.571505,1.272015,G2M,135,10044.0,14.0,CD14+ Mono,s1d1,NaN,...,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker,train
18025,1176,7.302143,0.771874,S,112,761.0,3.0,CD14+ Mono,s2d5,NaN,...,16710,40,27.8,O+,White,HISPANIC OR LATINO,Female,False,Smoker,train
18029,1251,9.355723,0.723323,G2M,115,750.0,5.0,CD14+ Mono,s3d7,NaN,...,11466,22,31.5,A+,Asian,NOT HISPANIC OR LATINO,Female,True,Nonsmoker,train
18030,1933,6.746421,1.748471,G2M,109,708.0,1.0,CD14+ Mono,s2d1,NaN,...,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker,train


In [21]:
test_X_df = test_data.layers['counts'].toarray()
pd.DataFrame(test_X_df).iloc[b,:]

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,28.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,39.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,38.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,76.0,0.0
18025,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0
18029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,41.0,0.0
18030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,34.0,0.0


In [22]:
test_X_df = test_data.X.toarray()
pd.DataFrame(test_X_df).iloc[b,:]

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,6.564087,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.720734,0.000000,0.0,0.0,20.180548,0.0
6,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,46.966347,0.0
9,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.797515,0.000000,0.000000,0.0,0.0,31.103073,0.0
17,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.996492,0.996492,0.000000,0.000000,0.0,0.0,37.866688,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18023,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.786155,0.000000,0.000000,0.0,0.0,59.747742,0.0
18025,0.0,1.295549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.295549,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,18.137688,0.0
18029,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,1.382508,0.000000,0.0,0.0,56.682838,0.0
18030,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.571928,0.000000,0.000000,1.143856,0.571928,0.0,0.0,19.445557,0.0


In [10]:
# Access the adata.X object, which contains a matrix of counts which has
# been already preprocessed, and the adata.layers[’counts’] object, which
# contains raw data. Plot histograms of both the raw data and the processed
# data. Pay attention to the X-axis and the range of values spanned by the data
test_raw_df = test_data.layers['counts'].toarray()

In [11]:
test_raw_df = pd.DataFrame(test_raw_df)
only_one_type = test_raw_df.iloc[b,:]

In [12]:
only_one_type

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,28.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,39.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,38.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,76.0,0.0
18025,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0
18029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,41.0,0.0
18030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,34.0,0.0


In [13]:
def get_median_of_summed_rows(df: pd.DataFrame):
    return df.sum(axis=1).median()

def get_mean_of_summed_rows(df: pd.DataFrame):
    return df.sum(axis=1).mean()

def get_min_of_summed_rows(df: pd.DataFrame):
    return df.sum(axis=1).min()

def get_max_of_summed_rows(df: pd.DataFrame):
    return df.sum(axis=1).max()

In [14]:
sets = [train_X_df, train_raw_df, test_X_df, test_raw_df]
mins = [get_min_of_summed_rows(pd.DataFrame(data)) for data in sets]
maxs = [get_max_of_summed_rows(pd.DataFrame(data)) for data in sets]
medians = [get_median_of_summed_rows(pd.DataFrame(data)) for data in sets]
means = [get_mean_of_summed_rows(pd.DataFrame(data)) for data in sets]
summary2 = pd.DataFrame(zip(mins, maxs, means, medians), index=['preprocessed train data', 'raw train data', 'preprocessed test data', 'raw test data'], columns=['min', 'max', 'mean', 'median'])
summary2

,min,max,mean,median
preprocessed train data,209.069855,25756926.0,17137.015625,1243.555054
raw train data,53.000000,57544.0,2210.539307,978.000000
preprocessed test data,239.688126,25756926.0,18288.505859,1243.269287
raw test data,72.000000,53853.0,2236.382812,984.000000
